In [13]:
import datasets
cache_path = '/data/kcl/lpy/hf'
data_path = "/data/kcl/lpy/data/HMMT_FEB_2025"
dataset = datasets.load_dataset(data_path, "default", cache_dir=cache_path)

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['problem_idx', 'problem', 'answer', 'problem_type'],
        num_rows: 30
    })
})

In [15]:
system_prompt = '''Solve the following math problem step by step. The last line of your response should be of the form Answer: $Answer (without quotes) where $Answer is the answer to the problem.\n\n'''

In [16]:
    train_dataset = dataset["train"]
    def make_map_fn(split):
        def process_fn(example, idx):
            extra_info = {'index':example['problem_idx']}
            extra_info["need_tools_kwargs"] = True
            extra_info["tools_kwargs"] = {
                "code_interpreter": {
                    "create_kwargs": {
                        "ground_truth": example["answer"],
                    },
                },
            }
            extra_info['split'] = None
            extra_info['raw_problem'] = example['problem']
            example["extra_info"] = extra_info
            example['data_source'] = 'aime_hmmt_feb_2025'
            example['ability'] = 'MATH'
            example['reward_model'] = {
                'ground_truth': example["answer"],
                'style': 'rule-lighteval/MATH_v2'
            }
            example['prompt'] = [{
                'content' : f'{system_prompt}{example['problem']}\n\nRemember to put your answer on its own line after "Answer:".',
                'role' : 'user'
            }]
            return example
        return process_fn

    train_dataset = train_dataset.map(function=make_map_fn("train"), with_indices=True)
    train_dataset = datasets.concatenate_datasets([train_dataset]*64).shuffle(seed=7)

Map: 100%|██████████| 30/30 [00:00<00:00, 2891.09 examples/s]


In [19]:
    import os
    train_dataset.to_parquet(os.path.join(data_path, "test_verl.parquet"))

Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00,  8.00ba/s]


2657088

In [18]:
__index_level_0__list = []
for item in train_dataset:
    __index_level_0__list.append(item['__index_level_0__'])

In [23]:
len(set(__index_level_0__list))

960

In [21]:
len(train_dataset)

960

In [13]:
datasets.__version__

'4.5.0'